![dvd_image](dvd_image.jpg)

A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Import any additional modules and start coding below

In [2]:
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler

In [3]:
# Load the dataset
df = pd.read_csv('rental_info.csv')

# Convert 'rental_date' and 'return_date' to datetime
df['rental_date'] = pd.to_datetime(df['rental_date'])
df['return_date'] = pd.to_datetime(df['return_date'])

# Create 'rental_length_days' column
df['rental_length_days'] = (df['return_date'] - df['rental_date']).dt.days

# Create dummy variables for 'special_features'
df['deleted_scenes'] = (df['special_features'].str.contains('Deleted Scenes')).astype(int)
df['behind_the_scenes'] = (df['special_features'].str.contains('Behind the Scenes')).astype(int)

# Select features for X and y
X = df[['amount', 'amount_2', 'rental_rate', 'rental_rate_2', 'release_year', 
        'length', 'length_2', 'replacement_cost', 'NC-17', 'PG', 'PG-13', 'R', 
        'deleted_scenes', 'behind_the_scenes']]
y = df['rental_length_days']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=9)

# Normalize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Model selection and evaluation
models = {
    'Linear Regression': LinearRegression(),
    'Ridge': Ridge(),
    'Random Forest': RandomForestRegressor(random_state=9)
}

best_model = None
best_mse = float('inf')

for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    mse = mean_squared_error(y_test, y_pred)
    print(f'{name} MSE: {mse}')
    
    if mse < best_mse:
        best_mse = mse
        best_model = model

# Checking if the best MSE meets the requirement
if best_mse < 3:
    print("Recommended model meets the MSE requirement.")
else:
    print("No model meets the MSE requirement of less than 3.")

# Output the best model and its MSE
print("Best Model:", type(best_model).__name__)
print("Best MSE:", best_mse)

Linear Regression MSE: 2.9417238646975976
Ridge MSE: 2.9418707029214026
Random Forest MSE: 2.0302178088293976
Recommended model meets the MSE requirement.
Best Model: RandomForestRegressor
Best MSE: 2.0302178088293976
